In [1]:
!pip install -q langchain langchain_huggingface langchain-community faiss-cpu accelerate einops transformers datasets sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.

## Setup

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_key = user_secrets.get_secret("HF_Token")


import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_key

## Import Libraries

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Markdown, display
from datasets import load_dataset

## Load and Prepare the MedQuad Dataset

In [4]:
# Load the first 20,000 examples from LavitA/MedQuad
dataset = load_dataset("LavitA/MedQuad", split="train[:20000]")

# Drop rows with missing question or answer
dataset = dataset.filter(lambda row: row["question"] is not None and row["answer"] is not None)

# Convert to langchain Documents
documents = [
    Document(page_content=f"Question: {row['question']}\nAnswer: {row['answer']}")
    for row in dataset
]

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

(…)-00000-of-00001-e36383d177026d53.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47441 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [5]:
documents[0]

Document(metadata={}, page_content='Question: What is (are) keratoderma with woolly hair ?\nAnswer: Keratoderma with woolly hair is a group of related conditions that affect the skin and hair and in many cases increase the risk of potentially life-threatening heart problems. People with these conditions have hair that is unusually coarse, dry, fine, and tightly curled. In some cases, the hair is also sparse. The woolly hair texture typically affects only scalp hair and is present from birth. Starting early in life, affected individuals also develop palmoplantar keratoderma, a condition that causes skin on the palms of the hands and the soles of the feet to become thick, scaly, and calloused.  Cardiomyopathy, which is a disease of the heart muscle, is a life-threatening health problem that can develop in people with keratoderma with woolly hair. Unlike the other features of this condition, signs and symptoms of cardiomyopathy may not appear until adolescence or later. Complications of c

## Embedding and FAISS Vector Store

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

faiss_db = FAISS.from_documents(documents, embedding_model)

2025-05-27 18:23:04.544405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748370184.748771      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748370184.809697      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Build the Retrieval System

In [7]:
def retriev_docs(query, k=5):
    docs = faiss_db.similarity_search(query, k)
    return docs

# Test it
query = "What causes asthma?"
docs_faiss = retriev_docs(query)
docs_faiss

[Document(id='571fae95-2832-4ae4-a0fe-5f4e6987adc3', metadata={}, page_content='Question: What causes Asthma ?\nAnswer: The exact cause of asthma isn\'t known. Researchers think some genetic and environmental factors interact to cause asthma, most often early in life. These factors include:\n                \nAn inherited tendency to develop allergies, called atopy (AT-o-pe)\n                \nParents who have asthma\n                \nCertain respiratory infections during childhood\n                \nContact with some airborne allergens or exposure to some viral infections in infancy or in early childhood when the immune system is developing\n                \nIf asthma or atopy runs in your family, exposure to irritants (for example, tobacco smoke) may make your airways more reactive to substances in the air.\n                \nSome factors may be more likely to cause asthma in some people than in others. Researchers continue to explore what causes asthma.\n                \nThe "Hyg

## Build the Generative System

In [8]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation"
)

chat_model = ChatHuggingFace(llm=llm)

In [9]:
def ask(query):
    messages = [
        SystemMessage(content="You are a Doctor"),
        HumanMessage(content=query)
    ]

    response = chat_model.invoke(messages)
    display(Markdown(response.content))

# Test
ask("What causes asthma?")

 Asthma is a chronic disease characterized by inflammation and narrowing of the airways, leading to difficulty breathing, coughing, wheezing, and shortness of breath. The exact cause of asthma is not fully understood but is believed to be a combination of genetic and environmental factors.

Several factors are known to contribute to the development and exacerbation of asthma, including:

1. Genetics: Individuals with a family history of asthma are more likely to develop the condition themselves, indicating a genetic component to the disease.

2. Allergens: Common allergens such as pollen, mold, dust mites, and pet dander can trigger an immune response in the airways, leading to inflammation and asthma symptoms.

3. Respiratory infections: Viral respiratory infections, such as the common cold or flu, can cause inflammation and narrowing of the airways, resulting in asthma symptoms.

4. Exposure to pollutants: Environmental pollutants, such as tobacco smoke, chemical fumes, and air pollution, can irritate the airways and worsen asthma symptoms.

5. Physical activity: Exercise-induced asthma occurs when vigorous physical activity triggers an immune response in the airways, causing them to narrow and result in asthma symptoms.

6. Occupational hazards: Exposure to certain occupational hazards, such as chemicals or dust in the workplace, can worsen asthma symptoms in some individuals.

7. Stress and allergens: Emotional stress and reactions to allergens can also exacerbate asthma symptoms.

In conclusion, asthma is caused by a complex interplay of genetic and environmental factors, with common triggers including allergens, respiratory infections, pollutants, physical activity, occupational hazards, stress, and reactions to allergens. Effective management of asthma involves avoiding triggers, taking prescribed medications, and using asthma action plans to monitor symptoms and avoid exacerbation.

## Define RAG Functions

In [11]:
import re
from langchain.schema import SystemMessage, HumanMessage
from IPython.display import Markdown, display

# Text cleaner function
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove non-ASCII characters
    return text.strip()

# Improved ask2 function with better prompt
def ask2_improved(query, context):
    prompt = f"""You are a medical expert. Use the following medical information to answer the question comprehensively.

Medical Information:
{context}

Question: {query}

Instructions:
- Provide a detailed, helpful answer using the medical information above
- Organize your response clearly with relevant details
- If the information is incomplete, use what's available and note any limitations
- Only say "I don't know" if the context contains no relevant information at all"""

    messages = [
        SystemMessage(content=prompt),
        HumanMessage(content=query)
    ]
    response = chat_model.invoke(messages)
    return response.content

# 📚 Improved RAG function
def ask_rag_improved(query, k=5):
    # Get documents
    docs = retriev_docs(query, k=k)
    print(f"📚 Retrieved {len(docs)} documents")
    
    # Clean and combine context
    cleaned_contexts = [clean_text(doc.page_content) for doc in docs]
    combined_context = "\n\n---\n\n".join(cleaned_contexts)  # Better separator
    
    # Increase context limit for better coverage
    MAX_CONTEXT_CHARS = 6000
    safe_context = combined_context[:MAX_CONTEXT_CHARS]
    
    print(f"📄 Context length: {len(safe_context)} characters")
    
    # Get response
    response = ask2_improved(query, safe_context)
    return response



In [12]:
# 🧪 Alternative: Even more explicit prompt
def ask2_explicit(query, context):
    prompt = f"""As a doctor, answer this medical question using the provided information.

The following medical documents contain information relevant to the question:

{context}

Question: {query}

Please provide a comprehensive answer based on the information above. Extract and synthesize the relevant details to give a complete response."""

    messages = [
        SystemMessage(content=prompt),
        HumanMessage(content=query)
    ]
    response = chat_model.invoke(messages)
    return response.content

def ask_rag_explicit(query, k=5):
    docs = retriev_docs(query, k=k)
    cleaned_contexts = [clean_text(doc.page_content) for doc in docs]
    combined_context = "\n\n".join(cleaned_contexts)
    safe_context = combined_context[:6000]
    
    response = ask2_explicit(query, safe_context)
    return response


# 🔍 Debug function to see what's happening
def debug_rag_detailed(query):
    print(f"🔍 Query: {query}")
    
    docs = retriev_docs(query, k=5)
    print(f"📚 Retrieved {len(docs)} documents")
    
    cleaned_contexts = [clean_text(doc.page_content) for doc in docs]
    combined_context = "\n\n---\n\n".join(cleaned_contexts)
    safe_context = combined_context[:6000]
    
    print(f"\n📄 Context Preview (first 500 chars):")
    print("="*50)
    print(safe_context[:500] + "...")
    print("="*50)
    
    # Test both approaches
    print(f"\n🤖 Original approach:")
    original_response = ask2(query, safe_context)
    print(original_response)
    
    print(f"\n🤖 Improved approach:")
    improved_response = ask2_improved(query, safe_context)
    print(improved_response)
    
    return improved_response

In [13]:
# Test functions
print("Testing improved RAG...")
query = "What causes asthma?"

# Method 1: Improved prompt
print("\n" + "="*60)
print("METHOD 1: Improved Prompt")
print("="*60)
response1 = ask_rag_improved(query)
display(Markdown(response1))

print("\n" + "="*60) 
print("METHOD 2: Explicit Prompt")
print("="*60)
response2 = ask_rag_explicit(query)
display(Markdown(response2))

print("\n" + "="*60)
print("DEBUG: Detailed Analysis")
print("="*60)
debug_response = debug_rag_detailed(query)

Testing improved RAG...

METHOD 1: Improved Prompt
📚 Retrieved 5 documents
📄 Context length: 6000 characters


 Asthma is a complex chronic respiratory condition characterized by the inflammation and narrowing of airways in the lungs, leading to symptoms like wheezing, chest tightness, shortness of breath, and coughing. The following factors contribute to the development of asthma, as detailed in various research findings and medical opinions presented above:

1. Genetic predisposition: Some people exhibit a genetic tendency to develop asthma, often called atopy. This likelihood of developing asthma often exists in individuals who have one or both parents with the condition.

2. Environmental factors:
   a. Atopic history: Individuals with a personal or family history of allergies such as hay fever or eczema are more likely to develop asthma. 
   b. Infections: Respiratory infections during infancy or early childhood and exposure to certain viruses can influence the development of asthma, probably by affecting immune system maturation.
   c. Airway reactivity: Environmental triggers like pollutants, smoke, dust mites, pet dander, or pollen can cause airway inflammation and lead to asthma symptoms.

3. The "Hygiene Hypothesis": This theory posits that Western lifestyle changes leading to more sanitation and less early-life exposure to various microorganisms might alter the natural development of an immune system and increase the risk of allergic diseases like asthma.

4. The atopic march: In some individuals, the progression of atopic diseases begins with eczema and proceeds to food allergies, hay fever, and ultimately asthma. Not everyone with eczema or food allergies will develop asthma, and similarly, not all asthma sufferers have progressed through the atopic march.

5. Bronchial hyperresponsiveness: A hallmark of asthma brought on by the immune response to allergens, bronchial hyperresponsiveness results in more severe and persistent airway constriction upon exposure to asthma triggers, which can exacerbate asthma symptoms.

6. Environmental lifestyle: Factors like obesity and exposure to tobacco smoke can increase asthma vulnerability and influence the severity of asthma attacks.

Asthma treatment generally entails long-term usage of medications to alleviate symptoms and prevent attacks. These medications can be anti-inflammatory, relieving bronchial spasm, or provide fast-acting relief during acute asthma episodes. Additionally, identifying and minimizing exposure to specific asthma triggers can help manage the condition. Considering the multifactorial nature of asthma, a comprehensive approach that addresses both genetic and environmental factors is essential for effective asthma management.


METHOD 2: Explicit Prompt


 Asthma has a complex etiology that involves interactions between genetic and environmental factors, often presenting early in life. The development of asthma is polygenic, suggesting that individuals carry certain genetic predispositions, such as a tendency to develop allergic conditions—atopy. The condition can also result from acute or recurrent exposure to various irritants known to trigger respiratory issues.

Key contributors to asthma development include:

1. Genetic Predisposition: A genetic tendency toward atopy (having an increased likelihood of developing allergies) can be inherited from parents, especially when asthma is prevalent in the family.

2. Early Life Factors: In early biological development, factors such as respiratory infections or exposure to specific allergens can predispose the immune system toward an asthmatic response.

3. Atopy: If atopy isn't managed during critical periods of immune system development, it can increase the probability of asthma.

4. Environmental Irritants and Allergens: Contact with irritants like tobacco smoke, and pollen, dust, animal dander, and mold can exacerbate or trigger asthma by provoking the immune system.

5. Hygiene Hypothesis: This theory posits that increased cleanliness and reduced exposure to childhood infections in Western societies may lead to under-stimulation of the immune system. The "missing" exposures can result in the premature maturation of the immune system, making it more susceptible to allergies and asthma.

6. Lifestyle and Environmental Factors: Ongoing exposure to allergens and other irritants in certain living conditions can maintain or exacerbate asthma.

It's also essential to consider that asthma is multifactorial; therefore, it's not productive to pinpoint a single cause. Instead, its pathogenesis likely involves a combination of genetic susceptibility and various environmental triggers that collectively lead to the hyperresponsiveness of the airways and recurrent asthma attacks. Preventive measures, such as avoiding allergens and irritants and maintaining a clean environment, are crucial for individuals with a genetic predisposition or atopy to manage or prevent the onset of asthma.


DEBUG: Detailed Analysis
🔍 Query: What causes asthma?
📚 Retrieved 5 documents

📄 Context Preview (first 500 chars):
Question: What causes Asthma ? Answer: The exact cause of asthma isn't known. Researchers think some genetic and environmental factors interact to cause asthma, most often early in life. These factors include: An inherited tendency to develop allergies, called atopy (AT-o-pe) Parents who have asthma Certain respiratory infections during childhood Contact with some airborne allergens or exposure to some viral infections in infancy or in early childhood when the immune system is developing If asth...

🤖 Original approach:
 The exact cause of asthma isn't known, but researchers think that a combination of genetic and environmental factors contributes to it. Some people may be born with a genetic predisposition to developing asthma, which is referred to as having an atopic background. Atopic means having a genetically predisposed tendency to develop an allergic response. Env

## Save the FAISS vector store

In [17]:
faiss_db.save_local("faiss_medquad_index")

## Inference

In [16]:
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 32.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [18]:
import re
import gradio as gr
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain.vectorstores.faiss import FAISS
from langchain.schema import SystemMessage, HumanMessage


# Text Cleaner
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove non-ASCII characters
    return text.strip()

# Load embedding model and FAISS DB
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

faiss_db = FAISS.load_local(
    "/kaggle/working/faiss_medquad_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Retrieval function
def retriev_docs(query, k=5):
    return faiss_db.similarity_search(query, k)

# LLM setup (Phi-3)
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation"
)
chat_model = ChatHuggingFace(llm=llm)

In [ ]:
def ask2_improved(query, context):
    prompt = f"""You are a medical expert. Use the following medical information to answer the question comprehensively.

Medical Information:
{context}

Question: {query}

Instructions:
- Provide a detailed, helpful answer using the medical information above
- Organize your response clearly with relevant details
- If the information is incomplete, use what's available and note any limitations
- Only say "I don't know" if the context contains no relevant information at all"""

    messages = [
        SystemMessage(content=prompt),
        HumanMessage(content=query)
    ]
    response = chat_model.invoke(messages)
    return response.content

# Main RAG function
def ask_rag_improved(query, k=5):
    docs = retriev_docs(query, k=k)
    cleaned_contexts = [clean_text(doc.page_content) for doc in docs]
    combined_context = "\n\n---\n\n".join(cleaned_contexts)
    safe_context = combined_context[:6000]
    return ask2_improved(query, safe_context)

# Gradio Interface
def rag_gradio_infer(radio_question, custom_question):
    query = custom_question.strip() if custom_question.strip() else radio_question
    if not query:
        return "❗ Please enter or select a question."
    return ask_rag_improved(query)

# UI Setup
predefined_questions = [
    "What causes asthma?",
    "What are the genetic changes related to keratoderma with woolly hair?",
    "What are the symptoms of diabetes?",
    "How does hypertension affect the body?",
    "What is the treatment for migraine?"
]

with gr.Blocks() as demo:
    gr.Markdown("## 🩺 Doctor AI - Medical Assistant (RAG)")
    gr.Markdown("Choose a predefined medical question or write your own:")

    with gr.Row():
        radio = gr.Radio(predefined_questions, label="Predefined Questions", interactive=True)

    custom_input = gr.Textbox(label="Custom Question", placeholder="E.g., What are the symptoms of pneumonia?")
    submit_btn = gr.Button("🔍 Get Medical Answer")
    output = gr.Markdown()

    submit_btn.click(fn=rag_gradio_infer, inputs=[radio, custom_input], outputs=output)

# Launch
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://27fcd36ef23c553c09.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
